In [1]:
from GG_mthesis import *
%matplotlib inline

In [2]:
sns.set_style('whitegrid')
sns.set_palette('colorblind')
sns.set(font_scale=1.2)

In [3]:
#define the path
path = '.\\data\\01_raw\\tsg_orchestra\\b1903933.nc'

#open the dataset with the defined path
ds_orig = xr.open_dataset(path) 

#Squeeze eliminates dimensions of length 0 and 1. Drop REFMAX as it is not needed. Swap MAXT (count) with TIME 
#to make TIME unique dimension (MAXT is a count a not needed, you can access the same with .isel). Rename coords to
#my standard.
ds = ds_orig.squeeze().drop_dims('REFMAX').swap_dims({'MAXT':'TIME'}).rename({
    'LONGITUDE':'lon','LATITUDE':'lat','TIME':'time','DEPTH':'depth'})

#Definition of new time attributes (datetime). Retrieved from original dataset and modified.
new_time_attr = {'long_name': 'Chronological DateTime Date',
 'standard_name': 'time',
 'ancillary_variables': 'TIME_SEADATANET_QC',
 'axis': 'T',
 'valid_min': 0.0,
 'valid_max': 2500000.0,
 'sdn_parameter_urn': 'SDN:P01::CJDY1101',
 'sdn_parameter_name': 'DateTime (chronological)',
 'sdn_uom_name': 'Days',
 'sdn_uom_urn': 'SDN:P06::UTAA'}

#Pull out the index (a CFTimeIndex) and use built in to_datetimeindex method.
datetimeindex = ds.indexes['time'].to_datetimeindex()

#Replace CFTimeIndex with DateTimeIndex:
ds['time'] = datetimeindex

#Attach new attributes
ds['time'].attrs = new_time_attr

C:\Users\gian_\AppData\Local\Temp/ipykernel_25972/4149106755.py:26: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = ds.indexes['time'].to_datetimeindex()


In [4]:
ds

<xarray.Dataset>
Dimensions:                 (time: 41277)
Coordinates:
    lat                     (time) float64 ...
    lon                     (time) float64 ...
    depth                   (time) float32 ...
  * time                    (time) datetime64[ns] 2017-11-08 ... 2017-12-06T1...
Data variables: (12/25)
    SDN_CRUISE              |S80 ...
    crs                     int32 ...
    SDN_EDMO_CODE           int32 ...
    SDN_STATION             |S80 ...
    SDN_LOCAL_CDI_ID        |S80 ...
    SDN_BOT_DEPTH           (time) float32 ...
    ...                      ...
    TEMPHU01_SEADATANET_QC  (time) float32 ...
    TEMPHU01                (time) float32 ...
    TEMPHU02_SEADATANET_QC  (time) float32 ...
    TEMPHU02                (time) float32 ...
    TMESSG01_SEADATANET_QC  (time) float32 ...
    TMESSG01                (time) float32 ...
Attributes:
    date_update:  2020-07-31T01:05:54+0100
    comment:      
    Conventions:  SeaDataNet_1.0 CF-1.6
    featureType:  trajectory
    title:        SeaDataNet NETCDF TRAJECTORY Climate Forecast 1.6 - GENERAT...

In [4]:
#This dataset has been reshaped
path_to = '.\\data\\02_intermediate\\tsg_orchestra\\b1903933_1_reshaped.nc'
ds.to_netcdf(path_to)